# Voiceshell Demo

#### Imports

In [1]:
import math
import random
from __future__ import unicode_literals
import spacy
import os
import csv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import wave, sys, pyaudio
import speech_recognition as sr

#### Creating/defining basic functions

In [2]:
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0
    
def sentvec(s):
    sent = nlp(s)
    word_vectors = [w.vector for w in sent]
    return meanv(word_vectors)

def spacy_closest_sent(space, input_str, n=1):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec),
                  reverse=True)[:n]

#### Set corpus file here:

In [3]:
nlp = spacy.load('en')

corpusFile = 'revisedCorpus/voiceshell_audio_LUT.csv'

In [4]:
with open(corpusFile, 'r') as f:
    reader = csv.reader(f)
    all_lines = list(reader)
    
audio_lookup_table = {}
sentences = []
for line in all_lines:
    sentence = line[0]
    if not sentence.isspace():
        sentences.append(nlp(sentence))
    filename = line[1]
    audio_lookup_table[sentence] = filename

In [5]:
prompts = ["How are you feeling?", "Tell me a quote you like.", "What are the first three words that pop into your head?", "What day is it today?", "What's your favourite colour?", "Where are you right now?", "What's your job?", "What do you like to eat?", "What animal would you want as a pet?"]

In [6]:
# get microphone input
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Just a moment, calibrating the microphone...")
    # 5 second listen and find ambient noise level
    r.adjust_for_ambient_noise(source, duration=5)
    inquiry = random.choice(inquiries)
    
    # read question aloud and print it
    os.system("say '" + inquiry + "'")
    print(inquiry," (Speak aloud, please.)")
    audio = r.listen(source)
    
# recognise speech with Sphinx
try:
    recogSpeech = r.recognize_sphinx(audio) + "'"
    print("I heard you say: ", recogSpeech)
    for sent in spacy_closest_sent(sentences, recogSpeech):
        print("----------")
        print("...")
        print(sent.text)
        print("----------")
except sr.UnknownValueError:
    print("Sorry, I couldn't understand you.")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

Just a moment, calibrating the microphone...


NameError: name 'inquiries' is not defined

In [14]:
def runLoop():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Just a second, calibrating the microphone...")
        r.adjust_for_ambient_noise(source, duration=5)
        print("Say something!")
        audio = r.listen(source)
        
    try:
        userInput = r.recognize_sphinx(audio)
        print(userInput)
    
        if(userInput == 'open the pod bay doors'):
            audioFile = 'PoemsAudio/sorry.wav'
        else:    
            for sent in spacy_closest_sent(sentences, userInput):
                print(sent.text)

            audioFile = audio_lookup_table[sent.text]
            print(audioFile)

        sound = wave.open(audioFile)
        p = pyaudio.PyAudio()
        chunk = 1024
        stream = p.open(format = p.get_format_from_width(sound.getsampwidth()), channels = sound.getnchannels(), rate = sound.getframerate(), output = True)
        data = sound.readframes(chunk)
        while len(data) > 0:
            stream.write(data)
            data = sound.readframes(chunk)
        stream.stop_stream()
        stream.close()

        p.terminate
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand you.")
    except sr.RequestError as e:
        print("Sphinx error; {0}".format(e))

## Run here:

In [23]:
runLoop()

Just a second, calibrating the microphone...
Say something!
of a pod bay doors
open the pod bay doors
PoemsAudio/sorry.mp3


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


FileNotFoundError: [Errno 2] No such file or directory: 'PoemsAudio/sorry.mp3'